In [0]:
%fs ls /FileStore/


path,name,size,modificationTime
dbfs:/FileStore/corr_matrix.xlsx,corr_matrix.xlsx,5157,1751113691319
dbfs:/FileStore/pubmed/,pubmed/,0,1751047775134
dbfs:/FileStore/pubmed_baseline/,pubmed_baseline/,0,1750500080279
dbfs:/FileStore/pubmed_filtrado/,pubmed_filtrado/,0,1750502495489
dbfs:/FileStore/pubmed_filtrado_2020_2025/,pubmed_filtrado_2020_2025/,0,1750627256443
dbfs:/FileStore/pubmed_parsed_fast/,pubmed_parsed_fast/,0,1750530018462
dbfs:/FileStore/pubmed_parsed_fast_test/,pubmed_parsed_fast_test/,0,1750530594866
dbfs:/FileStore/schema_report.xlsx,schema_report.xlsx,7270,1751116013719
dbfs:/FileStore/schema_report_pubmeds_mapeadas.xlsx,schema_report_pubmeds_mapeadas.xlsx,6922,1751124103695
dbfs:/FileStore/summary_report.xlsx,summary_report.xlsx,5777,1751107139741


📥 Descarga y procesamiento distribuido de archivos PubMed XML (.gz)
## Este notebook descarga archivos .xml.gz desde el FTP de PubMed en paralelo, los procesa usando Spark y guarda los datos como Parquet para análisis posteriores.

In [0]:
## esta se supone que es la mas optima

from ftplib import FTP
import os

# === CONFIGURACIÓN ===
ftp_host = "ftp.ncbi.nlm.nih.gov"
ftp_dir = "/pubmed/baseline/"
dbfs_target_dir = "/dbfs/FileStore/pubmed_baseline"

# Crear carpeta destino en DBFS si no existe
os.makedirs(dbfs_target_dir, exist_ok=True)

# === OBTENER LISTA DE ARCHIVOS DESDE FTP ===
ftp = FTP(ftp_host)
ftp.login()
ftp.cwd(ftp_dir)
all_files = [f for f in ftp.nlst() if f.endswith(".xml.gz")]
ftp.quit()

# Seleccionar archivos (puedes ajustar rango si no quieres todos)
selected_files = sorted([f for f in all_files if f.startswith("pubmed25n")])[:2000]

# === FILTRAR ARCHIVOS YA DESCARGADOS ===
existing_files = {f.name for f in dbutils.fs.ls("dbfs:/FileStore/pubmed_baseline")}
files_to_download = [f for f in selected_files if f not in existing_files]

print(f"📦 Total archivos a descargar: {len(files_to_download)}")

# === FUNCIÓN PARA EJECUTAR EN PARALELO (Spark Workers) ===
def download_ftp_file(file_name):
    import os
    from ftplib import FTP

    try:
        target_path = f"/dbfs/FileStore/pubmed_baseline/{file_name}"
        if os.path.exists(target_path):
            return f"✔️ {file_name} ya existe"

        ftp = FTP("ftp.ncbi.nlm.nih.gov")
        ftp.login()
        ftp.cwd("/pubmed/baseline/")
        with open(target_path, "wb") as f:
            ftp.retrbinary(f"RETR " + file_name, f.write)
        ftp.quit()
        return f"✅ {file_name} descargado"
    except Exception as e:
        return f"❌ Error {file_name}: {str(e)}"

# === DESCARGA DISTRIBUIDA USANDO SPARK ===
rdd = spark.sparkContext.parallelize(files_to_download, numSlices=100)
resultados = rdd.map(download_ftp_file).collect()

# Mostrar resumen
for r in resultados:
    print(r)


📦 Total archivos a descargar: 0


In [0]:
from pyspark.sql.functions import input_file_name


files = dbutils.fs.ls("dbfs:/FileStore/pubmed_baseline")

num_files = len(files)
total_size = sum(f.size for f in files)

print(f"Número de ficheros: {num_files}")
print(f"Tamaño total: {total_size/1024/1024:.2f} MB")



Número de ficheros: 1274
Tamaño total: 47220.08 MB


In [0]:
%fs ls /FileStore/pubmed_baseline/


path,name,size,modificationTime
dbfs:/FileStore/pubmed_baseline/pubmed25n0001.xml.gz,pubmed25n0001.xml.gz,19677806,1750500093775
dbfs:/FileStore/pubmed_baseline/pubmed25n0002.xml.gz,pubmed25n0002.xml.gz,17774034,1750500096161
dbfs:/FileStore/pubmed_baseline/pubmed25n0003.xml.gz,pubmed25n0003.xml.gz,16443024,1750500098427
dbfs:/FileStore/pubmed_baseline/pubmed25n0004.xml.gz,pubmed25n0004.xml.gz,18118733,1750500101202
dbfs:/FileStore/pubmed_baseline/pubmed25n0005.xml.gz,pubmed25n0005.xml.gz,17416603,1750500103596
dbfs:/FileStore/pubmed_baseline/pubmed25n0006.xml.gz,pubmed25n0006.xml.gz,20449707,1750500106166
dbfs:/FileStore/pubmed_baseline/pubmed25n0007.xml.gz,pubmed25n0007.xml.gz,20547744,1750500108677
dbfs:/FileStore/pubmed_baseline/pubmed25n0008.xml.gz,pubmed25n0008.xml.gz,19652303,1750500111118
dbfs:/FileStore/pubmed_baseline/pubmed25n0009.xml.gz,pubmed25n0009.xml.gz,9147705,1750500113149
dbfs:/FileStore/pubmed_baseline/pubmed25n0010.xml.gz,pubmed25n0010.xml.gz,12458196,1750500115356


%md
📥 Copia de ficheros archivos PubMed XML (.gz) filtrando aquellos entre 2020-2025
## Este notebook mira entre los archivos .xml.gz descargados y hace una copia de aquellos que tenga información de los años 2020-2025

In [0]:
import os
import re
import gzip
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
import shutil

# Ruta origen (DBFS local path)
source_dir = "/dbfs/FileStore/pubmed_baseline"
target_dir = "/dbfs/FileStore/pubmed_filtrado"
Path(target_dir).mkdir(parents=True, exist_ok=True)

# Compilar expresión regular para años
year_pattern = re.compile(r"\b(2020|2021|2022|2023|2024|2025)\b")

# Función para buscar año en el fichero
def contiene_anio(file_path):
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8', errors='ignore') as f:
            for i, line in enumerate(f):
                if year_pattern.search(line):
                    # Copiar a destino
                    dest_path = os.path.join(target_dir, os.path.basename(file_path))
                    shutil.copy(file_path, dest_path)
                    return f"✅ {os.path.basename(file_path)} copiado"
                if i > 5000:  # Evitar leer archivos completos si no hace falta
                    break
        return f"⏭️ {os.path.basename(file_path)} ignorado"
    except Exception as e:
        return f"❌ Error {os.path.basename(file_path)}: {e}"

# Listar archivos
archivos = [os.path.join(source_dir, f.name) for f in dbutils.fs.ls("dbfs:/FileStore/pubmed_baseline") if f.name.endswith(".xml.gz")]

# Procesar en paralelo
with ThreadPoolExecutor(max_workers=32) as executor:
    results = list(executor.map(contiene_anio, archivos))

# Mostrar resumen
for r in results:
    print(r)


✅ pubmed25n0001.xml.gz copiado
✅ pubmed25n0002.xml.gz copiado
⏭️ pubmed25n0003.xml.gz ignorado
✅ pubmed25n0004.xml.gz copiado
⏭️ pubmed25n0005.xml.gz ignorado
✅ pubmed25n0006.xml.gz copiado
✅ pubmed25n0007.xml.gz copiado
✅ pubmed25n0008.xml.gz copiado
⏭️ pubmed25n0009.xml.gz ignorado
✅ pubmed25n0010.xml.gz copiado
✅ pubmed25n0011.xml.gz copiado
✅ pubmed25n0012.xml.gz copiado
✅ pubmed25n0013.xml.gz copiado
⏭️ pubmed25n0014.xml.gz ignorado
✅ pubmed25n0015.xml.gz copiado
✅ pubmed25n0016.xml.gz copiado
✅ pubmed25n0017.xml.gz copiado
✅ pubmed25n0018.xml.gz copiado
⏭️ pubmed25n0019.xml.gz ignorado
✅ pubmed25n0020.xml.gz copiado
✅ pubmed25n0021.xml.gz copiado
⏭️ pubmed25n0022.xml.gz ignorado
✅ pubmed25n0023.xml.gz copiado
✅ pubmed25n0024.xml.gz copiado
⏭️ pubmed25n0025.xml.gz ignorado
✅ pubmed25n0026.xml.gz copiado
✅ pubmed25n0027.xml.gz copiado
⏭️ pubmed25n0028.xml.gz ignorado
✅ pubmed25n0029.xml.gz copiado
✅ pubmed25n0030.xml.gz copiado
✅ pubmed25n0031.xml.gz copiado
✅ pubmed25n0032.xml.gz 

In [0]:
from pyspark.sql.functions import input_file_name


files = dbutils.fs.ls("dbfs:/FileStore/pubmed_filtrado")

num_files = len(files)
total_size = sum(f.size for f in files)

print(f"Número de ficheros: {num_files}")
print(f"Tamaño total: {total_size/1024/1024:.2f} MB")

Número de ficheros: 1154
Tamaño total: 45286.53 MB
